In [ ]:
import os
import pandas as pd;pd.__version__

In [ ]:
trade_folder = r'TradeOutput'
data_folder = r'DataOutput'
ticker = 'CL ##-##'

In [ ]:
def get_filter_time_frame(trade_time_frame):
    if trade_time_frame == 'Id3':
        return 'Id2'
    if trade_time_frame == 'Id2':
        return 'Id1'
    if trade_time_frame == 'Id1':
        return 'Daily'
    raise KeyError(trade_time_frame + 'is not supported.')

In [ ]:
df = pd.read_csv(os.path.join(data_folder, ticker +'.csv'), parse_dates=['TickDateTime']).set_index('TickDateTime')
df = df.loc[~df.index.duplicated(keep='first')]

In [ ]:
trade_df = pd.read_csv(os.path.join(trade_folder, ticker + '.csv'), names=['Direction', 'TimeFrame', 'Ticker', 'Quantity', 'EntryTime', 'EntryPrice', 'PeelTime', 'PeelPrice', 'StopPrice','TargetPrice','ATR','FtATR','BgMarketEnv','FtMarketEnv','BgVPOC','BgVAH','BgVAL','FtVPOC','FtVAH','FtVAL','NextBar1Price','NextBar2Price','NextBar3Price','MAE', 'MFE'], parse_dates=['EntryTime', 'PeelTime']);trade_df.head()

In [ ]:
for row_index, row in trade_df[trade_df['MFE'].isnull()].iterrows():
    trade_time_frame = row['TimeFrame'].lower().title()
    filter_time_frame = get_filter_time_frame(trade_time_frame)
    trade_entry_time = row['EntryTime']
    trade_entry_price = row['EntryPrice']
    trade_exit_time = row['PeelTime']
    
    # ATR of the trade
    _atr = df.loc[trade_entry_time, trade_time_frame + 'Atr']
    trade_df.loc[row_index, 'ATR'] = _atr
    trade_df.loc[row_index, 'FtATR'] = df.loc[trade_entry_time, filter_time_frame + 'Atr']
    
    # Background / Filter MarketEnv
    trade_df.loc[row_index, 'BgMarketEnv'] = df.loc[trade_entry_time, trade_time_frame + 'MarketEnv']
    trade_df.loc[row_index, 'FtMarketEnv'] = df.loc[trade_entry_time, filter_time_frame + 'MarketEnv']
    
    # Background / Filter VPOC
    trade_df.loc[row_index, 'BgVPOC'] = df.loc[trade_entry_time, trade_time_frame + 'VPOC']
    trade_df.loc[row_index, 'BgVAL'] = df.loc[trade_entry_time, trade_time_frame + 'VAL']
    trade_df.loc[row_index, 'BgVAH'] = df.loc[trade_entry_time, trade_time_frame + 'VAH']
    trade_df.loc[row_index, 'FtVPOC'] = df.loc[trade_entry_time, filter_time_frame + 'VPOC']
    trade_df.loc[row_index, 'FtVAL'] = df.loc[trade_entry_time, filter_time_frame + 'VAL']
    trade_df.loc[row_index, 'FtVAH'] = df.loc[trade_entry_time, filter_time_frame + 'VAH']
    
    # Stop Price of the trade based on ATR
    atr_ratio = 0.4
    stop_amount = _atr * atr_ratio
    if row['Direction'] == 'Long':
        stop_amount = -1 * stop_amount
    trade_df.loc[row_index, 'StopPrice'] = trade_entry_price + stop_amount
    
    # Target Price is based on Envelope price, if it is not as 1 to 1, then move to filter Envelope
    bg_envelope_top = df.loc[trade_entry_time, trade_time_frame + 'EnvelopeHigh']
    bg_envelope_bottom = df.loc[trade_entry_time, trade_time_frame + 'EnvelopeLow']
    ft_envelope_top = df.loc[trade_entry_time, filter_time_frame + 'EnvelopeHigh']
    ft_envelope_bottom = df.loc[trade_entry_time, filter_time_frame + 'EnvelopeLow']
    if row['Direction'] == 'Long':
        target_price = bg_envelope_top if (bg_envelope_top - trade_entry_price) >= stop_amount else ft_envelope_top
    else:
        target_price = bg_envelope_bottom if (trade_entry_price - bg_envelope_bottom) >= stop_amount else ft_envelope_bottom
    trade_df.loc[row_index, 'TargetPrice'] = target_price
    
    bg_bar_number_name = trade_time_frame + 'BarNumber'
    bg_bar_number = df.loc[trade_exit_time, bg_bar_number_name]
    
    # Next Close Bar 1
    trade_df.loc[row_index, 'NextBar1Price'] = df[df[bg_bar_number_name] == bg_bar_number + 1].iloc[-1].loc['ClosePrice']
    
    # Next Close Bar 2
    trade_df.loc[row_index, 'NextBar2Price'] = df[df[bg_bar_number_name] == bg_bar_number + 2].iloc[-1].loc['ClosePrice']
    
    # Next Close Bar 3
    trade_df.loc[row_index, 'NextBar3Price'] = df[df[bg_bar_number_name] == bg_bar_number + 3].iloc[-1].loc['ClosePrice']
    
    
    bg_entry_bar_number = df.loc[trade_entry_time, bg_bar_number_name]
    _df =  df[df[bg_bar_number_name].isin(list(range(bg_entry_bar_number, bg_bar_number + 4)))]
    min_value = _df['LowPrice'].min()
    max_value = _df['HighPrice'].max()
    if row['Direction'] == 'Long':
        trade_df.loc[row_index, 'MAE'] = min_value
        trade_df.loc[row_index, 'MFE'] = max_value
    else:
        trade_df.loc[row_index, 'MAE'] = max_value
        trade_df.loc[row_index, 'MFE'] = min_value
    
    
    
    

In [ ]:
trade_df.to_csv(ticker + '_processed.csv', index=False);trade_df.head()